# Train a deep CNN on XPS data on local machine

In this notebook, we will train a deep convolutional network on iron XPS spectra using the local CPU/GPU.

## Setup

### Install packages

In [ ]:
# Imports
import os
import datetime
import pytz
import numpy as np
import importlib

# Magic commands
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Run tensorflow on local CPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Disable tf warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf

### Load custom modules

In [ ]:
try:
    import importlib
    importlib.reload(classifier)
    importlib.reload(clfutils)
    print('Modules were reloaded.')
except:
    import network.classifier as classifier
    import network.utils as clfutils
    print('Modules were loaded.')

## Initial run

### Setting up the parameters & folder structure

In [ ]:
np.random.seed(502)
time =  datetime.datetime.now().astimezone(pytz.timezone('Europe/Berlin')).strftime("%Y%m%d_%Hh%Mm")
data_name = 'Fe_multiple_4_classes'

label_values = ['Fe metal', 'FeO', 'Fe3O4', 'Fe2O3']

clf = classifier.ClassifierMultiple(time = time,
                                    data_name = data_name,
                                    labels = label_values)

### Load and inspect the data

In [ ]:
input_filepath = r'C:\Users\pielsticker\Simulations\20200622_iron_linear_combination_1000000.h5'   
train_test_split = 0.2
train_val_split = 0.2
no_of_examples = 1000

X_train, X_val, X_test, y_train, y_val, y_test = clf.load_data_preprocess(input_filepath = input_filepath,
                                                                          no_of_examples = no_of_examples,
                                                                          train_test_split = train_test_split,
                                                                          train_val_split = train_val_split)
# Check how the examples are distributed across the classes:
class_distribution = clf.check_class_distribution()
clf.plot_class_distribution()
clf.plot_random(no_of_spectra = 9, dataset = 'train')  

### Design the model

In [ ]:
try:
    importlib.reload(models)
    print('Models module was reloaded.')
except:
    import network.models as models
    print('Models module was loaded.')

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, concatenate, Lambda
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import AveragePooling1D, MaxPooling1D
from tensorflow.keras.layers import LayerNormalization
from tensorflow.python.keras import backend as K

class CustomCNNMultiple(models.EmptyModel):
    def __init__(self, inputshape, num_classes):      
        input_1 = Input(shape = inputshape)
                
        conv_1_short = Conv1D(4, 5, padding = 'same',
                            activation = 'relu')(input_1)
        conv_1_medium = Conv1D(4, 10, padding = 'same',
                             activation = 'relu')(input_1)
        conv_1_long = Conv1D(4, 15, padding = 'same',
                           activation = 'relu')(input_1)
        sublayers = [conv_1_short, conv_1_medium, conv_1_long]
        merged_sublayers = concatenate(sublayers)
        
        conv_2 = Conv1D(4, 5, activation='relu')(merged_sublayers)
        average_pool_1 = AveragePooling1D()(conv_2)
        
        flatten_1 = Flatten()(average_pool_1)
        drop_1 = Dropout(0.2)(flatten_1)
        dense_1 = Dense(2000, activation = 'relu')(drop_1)
        
        dense_2 = Dense(num_classes, activation = 'sigmoid')(dense_1)
        
        output = Lambda(lambda x: x/K.sum(x), name = 'normalization')(dense_2)

        no_of_inputs = len(sublayers)

        super(CustomCNNMultiple, self).__init__(inputs = input_1,
                                                outputs = output,
                                                inputshape = inputshape,
                                                num_classes = num_classes,
                                                no_of_inputs = no_of_inputs,
                                                name = 'Custom_CNN_multiple')        

### Build the model

In [ ]:
clf.model = CustomCNNMultiple(clf.input_shape, clf.num_classes)

# Alternative: Build model from available models
# clf.model = models.CustomCNNMultiple(clf.input_shape, clf.num_classes)

### Compile and summarize the model

In [ ]:
from tensorflow.keras.optimizers import Adam
learning_rate = 1e-05
optimizer = Adam(learning_rate = learning_rate) 


def custom_loss(y_true, y_pred):
    """
    Custom loss for linear combination of XPS spectra.
    """
    squared_difference = tf.square(tf.subtract(y_true,y_pred))
    squared_output = tf.square(y_pred)
    
    return tf.reduce_sum(tf.multiply(squared_output,squared_difference))


#Compile model with custom loss function
clf.model.compile(loss = custom_loss, optimizer = optimizer)



# Plot summary and save model plot.
clf.summary()
clf.save_and_print_model_image()

### Training

In [ ]:
epochs = 10
batch_size = 32

hist = clf.train(checkpoint = True,
                 early_stopping = False,
                 tb_log = True, 
                 csv_log = True,
                 epochs = epochs, 
                 batch_size = batch_size,
                 verbose = 1)

### Graphs

In [ ]:
dir_name = clf.time + '_' + clf.data_name
graph = clfutils.TrainingGraphs(clf.history, dir_name) 
graph.plot_loss()

### Evaluation on test data

In [ ]:
test_loss = clf.evaluate()
print('Test loss: ' + str(test_loss))

### Prediction on test data

In [ ]:
pred_train, pred_test = clf.predict()

### Show some predictions

#### Training data

In [ ]:
clf.plot_random(no_of_spectra = 6, dataset = 'train', with_prediction = True)  

#### Test data

In [ ]:
clf.plot_random(no_of_spectra = 6, dataset = 'test', with_prediction = True)  

### Saving data

In [ ]:
clf.save_model()
clf.save_hyperparams()
clf.shelve_results(full = False)

### Report

In [ ]:
dir_name = clf.time + '_' + clf.data_name
rep = clfutils.Report(dir_name)  
rep.write()   

## Continue training 

In [ ]:
# Reload and train for more epochs
# model_path = r'C:\Users\pielsticker\Lukas\MPI-CEC\Projects\xpsdeeplearning\saved_models\20200608_17h51m_Fe_single_4_classes_CNN_simple' 
clf.load_model()
clf.model.compile(loss = 'categorical_crossentropy',
                  optimizer = optimizer, 
                  metrics = ['accuracy'])

new_learning_rate = 1e-04
epochs = 2
batch_size = 32
hist = clf.train(checkpoint = True,
                 early_stopping = False,
                 tb_log = True, 
                 csv_log = True,
                 epochs = epochs,
                 batch_size = batch_size, 
                 new_learning_rate = new_learning_rate) # Learning rate can be changed for retraining

from tensorflow.keras import backend as K
print('New learning rate: ' +\
      str(K.eval(clf.model.optimizer.lr)))

dir_name = clf.time + '_' + clf.data_name
graphs = clfutils.TrainingGraphs(hist, dir_name)
graph.plot_loss()

score = clf.evaluate()
test_loss = score[0]

pred_train, pred_test = clf.predict()
clf.plot_random(no_of_spectra = 6, dataset = 'train', with_prediction = True)  
clf.plot_random(no_of_spectra = 6, dataset = 'test', with_prediction = True)  

clf.save_model()
clf.save_hyperparams()
clf.shelve_results(full = False)  

rep = clfutils.Report(dir_name)  
rep.write()  


### Save output of notebook

In [ ]:
from IPython.display import Javascript, display
from nbconvert import HTMLExporter

def save_notebook():
    display(Javascript("IPython.notebook.save_notebook()"), include=['application/javascript'])

def output_HTML(read_file, output_file):
    import codecs
    import nbformat
    exporter = HTMLExporter()
    # read_file is '.ipynb', output_file is '.html'
    output_notebook = nbformat.read(read_file, as_version=4)
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, 'w', encoding='utf-8').write(output)

import time

save_notebook()
time.sleep(3)
current_file = 'train_local_mutliple.ipynb'
output_file = os.path.join(clf.log_dir,'train_local_mutliple_out.html')
output_HTML(current_file, output_file)